In [1]:
import bs4
import requests
import os
import numpy as np
import pandas as pd
from varios import utilerias

In [16]:
Home_manga = './static/Nuevos/'
smg_version = "1.3.2"

In [3]:
def get_headers():
        return {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/37.0.2062.94 Chrome/37.0.2062.94 Safari/537.36'}

In [4]:
url_base = 'https://www.manhwas.net/biblioteca?page={}'

In [5]:
def get_capitulos(url_del_manga) -> list:
    print(f"url del manga: {url_del_manga}")
    # Se obtienen los números de capítulos y sus urls
    resultado = requests.get(url_del_manga)
    sopa = bs4.BeautifulSoup(resultado.text, 'lxml')
    capitulos = sopa.select('.fa-book')
    listado = []

    for capitulo in capitulos:
        titulo = capitulo.select('p')[0]
        listado.append([titulo.text.strip(), capitulo['href']])

    return listado

In [33]:
def descargar_imagenes_del_capitulo(ruta_manga, url_del_capitulo, capitulo):
    Varios = utilerias.utilerias(smg_version)
    resultado = requests.get(url_del_capitulo)
    sopa = bs4.BeautifulSoup(resultado.text, 'lxml')
    capitulo_formateado = capitulo.replace('.', '-')
    capitulo_formateado = capitulo_formateado.replace('?', '')
    filtro = ruta_manga[ruta_manga.rindex('/')+1:]
    capitulo_formateado = capitulo_formateado.replace(filtro, '')
    capitulo_formateado = capitulo_formateado.strip()

    #imagenes = sopa.select(".tab-content")
    imagenes = sopa.find("div", {"id": "chapter_imgs"})
    # Se crean las carpetas respectivas para cada capítulo
    print(f'Creando {ruta_manga} y dentro {capitulo_formateado}')
    Varios.crear_carpeta_capitulo(ruta_manga, capitulo_formateado)
    # Se descargan las imagenes
    contador_imagen = 0
    for imagen in imagenes.select('img'):
            # Se extrae la imagen (url y nombre)
            # nombre_file = str(imagen['src'])
            # print(imagen['src'])
            nombre_file = str(contador_imagen).zfill(3) + '.jpg'
            if (imagen['src'] != '/discord.jpg'):
                # Se forma la ruta donde quedará finalmente
                ruta = os.path.join(ruta_manga, capitulo_formateado, nombre_file)
                ruta = os.path.abspath(ruta)
                
                # Se verifica si la imagen ya existía previamente
                if not(os.path.exists(ruta)):
                        # Se extrae la imagen de forma binaria de la web
                        imagen_save = requests.get(imagen['src'])
                
                        # Se guarda la imagen en disco
                        f = open (ruta, 'wb')
                        f.write(imagen_save.content)
                        f.close()
            contador_imagen = contador_imagen + 1

In [38]:
listado = []

for pagina in range(1, 2):
    url_pagina = url_base.format(pagina)
    resultado = requests.get(url_pagina)
    sopa = bs4.BeautifulSoup(resultado.text, 'lxml')
    manhwas = sopa.select('.anime')
    for manhwa in manhwas:
        texto = manhwa.select('h3')[0].getText()
        url = manhwa.select('a')[0]['href']
        item = (texto, url)
        listado.append(item)

        #print(f"Nombre: {texto} url: {url}")
arreglo = np.array(listado)

# Se inicia utilerías
Varios = utilerias.utilerias(smg_version)

# print(arreglo[0][0]) # [0] Nombre [1] url
for manga, url in arreglo:
    print(manga)
    ruta_manga = Varios.crear_carpeta_manga(Home_manga, manga)
    listado_capitulos = get_capitulos(url)
    listado_capitulos.sort()
    for capitulo, url in listado_capitulos:
        print(capitulo)
        descargar_imagenes_del_capitulo(ruta_manga, url, capitulo)



# df = pd.DataFrame(arreglo, columns = ['nombre','url'])
# df.to_csv('listado.csv', encoding='utf-8-sig', sep=';', index=False)

La Chica del Pan
url del manga: https://www.manhwas.net/manga/la-chica-del-pan
La Chica del Pan Capítulo 3.00
Creando ./static/Nuevos/La Chica del Pan y dentro Capítulo 3-00
La Chica del Pan Capítulo 2.00
Creando ./static/Nuevos/La Chica del Pan y dentro Capítulo 2-00
La Chica del Pan Capítulo 1.00
Creando ./static/Nuevos/La Chica del Pan y dentro Capítulo 1-00
My harem is entirely female girls
url del manga: https://www.manhwas.net/manga/my-harem-is-entirely-female-girls
My harem is entirely female girls Capítulo 14.00
Creando ./static/Nuevos/My harem is entirely female girls y dentro Capítulo 14-00
My harem is entirely female girls Capítulo 13.00
Creando ./static/Nuevos/My harem is entirely female girls y dentro Capítulo 13-00
My harem is entirely female girls Capítulo 12.00
Creando ./static/Nuevos/My harem is entirely female girls y dentro Capítulo 12-00
My harem is entirely female girls Capítulo 11.00
Creando ./static/Nuevos/My harem is entirely female girls y dentro Capítulo 11-00

OSError: [WinError 123] El nombre de archivo, el nombre de directorio o la sintaxis de la etiqueta del volumen no son correctos: 'e:\\Sw\\smg\\static\\Nuevos\\Choppiri Ecchi na Sanshimai demo, Oyomesan ni Shitekuremasuka?'

In [24]:
listado_capitulos.sort()

In [31]:
listado_capitulos[1]

['La Chica del Pan Capítulo 2.00',
 'https://www.manhwas.net/leer/la-chica-del-pan-2.00']